In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
select name,type from sqlite_master
where type in ('table','view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


A query to find the genres that sell the most tracks in USA. 

In [19]:
%%sql
with track_genre as
(
    select 
        t.track_id,
        g.genre_id,
        g.name
    from track t
    inner join genre g
    on t.genre_id = g.genre_id
)

select 
    g.name,
    COUNT(il.invoice_id) as number_of_purchases,
    CAST(COUNT(il.invoice_id) as Float)/(select COUNT(invoice_line_id) from invoice_line) *100 as percentage
from track_genre g
inner join invoice_line il
on g.track_id = il.track_id
group by g.name
order by number_of_purchases desc;

Done.


name,number_of_purchases,percentage
Rock,2635,55.3920538154299
Metal,619,13.012402774858103
Alternative & Punk,492,10.342652932520496
Latin,167,3.510615934412445
R&B/Soul,159,3.3424427159974774
Blues,124,2.606684885431995
Jazz,121,2.543619928526382
Alternative,117,2.4595333193188984
Easy Listening,74,1.5556022703384484
Pop,63,1.3243640950178683


Looking at the most sold genres in the USA, we can conclude that the most tracks sold are Rock, Heavy Metal and Alternative & Punk.

A query to find the total dollar amount of sales assigned to each Sales support agent, along with neccessary extra attributes to analyse performance

In [103]:
%%sql
select 
    t.employee_name,
    SUM(i.total_purchases) as sales_per_employee,
    COUNT(i.customer_id) as total_customers,
    CAST(COUNT(i.customer_id) as Float)/(select COUNT(customer_id) from customer) * 100 customer_percentage,
    SUM(i.total_purchases)/(select SUM(total) from invoice) * 100 sales_percentage,
    SUM(i.total_purchases)/COUNT(i.customer_id) as avg_sales_per_customer
from (
    select c.customer_id,
        SUM(i.total) as total_purchases
    from customer c
    inner join invoice i on c.customer_id = i.customer_id
    group by c.customer_id
) i
inner join (
    select e.first_name || " " || e.last_name as employee_name,
    c.customer_id
    from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    ) t
on i.customer_id = t.customer_id
group by t.employee_name
order by sales_per_employee desc;

Done.


employee_name,sales_per_employee,total_customers,customer_percentage,sales_percentage,avg_sales_per_customer
Jane Peacock,1731.5099999999998,21,35.59322033898305,36.76686987597229,82.45285714285713
Margaret Park,1584.0000000000002,20,33.89830508474576,33.634643682993534,79.20000000000002
Steve Johnson,1393.92,18,30.508474576271187,29.598486441034304,77.44


Looking at the returned tuples, We can say that 'Jane Peacock' has had most customers as well as the maximum sales, the percentages are nearly  equal for 'Jane Peacock' and 'Margaret Park', whereas 'Steve Johnson' has lesser percentage of sales that customers.
With the above metrics we can conclude that 'Jane Peacock' is performing better that the rest. 'Margaret Park' is doing moderate and 'Steve Johnson' is underperforming. The avg_sales_per_customer column agrees with the conclusion

A query to find the sales in different countries, number of customers per country, and average customer purchase and order value. Countries having 1 customer are to be grouped as Others

In [154]:
%%sql
with 
country_avg as
(
    select
        DISTINCT(country),
        AVG(i.total) as avg_order
    from customer c
    inner join invoice i on c.customer_id = i.customer_id
    group by country
),
sales as
(
    select
        c.customer_id,
        SUM(i.total) as customer_purchases
    from customer c
    inner join invoice i
    on c.customer_id = i.customer_id
    group by c.customer_id
),
country_grouping as
(
    select 
        case
            when customers = 1 then 'Other'
            else country
        end as country,
        customers,
        sales,
        sales_per_customer,
        order_value
    from (
        select
            DISTINCT(c.country),    
            COUNT(c.customer_id) as customers,
            SUM(s.customer_purchases) as sales,
            SUM(s.customer_purchases)/COUNT(c.customer_id) as sales_per_customer,
            ca.avg_order as order_value
        from customer c
        inner join sales s
        on c.customer_id = s.customer_id
        inner join country_avg ca
        on c.country = ca.country
        group by c.country
    )
),
country_sorting as
(
    select 
        case
            when country = "Other" then 1
            else 0
        end as sort,
        country,
        SUM(customers) as number_of_customers,
        SUM(sales) as country_sales,
        AVG(sales_per_customer) as avg_sales_per_customer,
        AVG(order_value) as avg_order_value
    from country_grouping
    group by country
    order by sort asc,country_sales desc
)

select
    country,
    number_of_customers,
    country_sales,
    avg_sales_per_customer,
    avg_order_value
from country_sorting;

Done.


country,number_of_customers,country_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.03769230769231,7.942671755725189
Canada,8,535.59,66.94875,7.047236842105265
Brazil,5,427.67999999999995,85.53599999999999,7.0111475409836075
France,5,389.07,77.814,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24,136.62,9.108000000000002
United Kingdom,3,245.51999999999998,81.83999999999999,8.768571428571429
Portugal,2,185.13,92.565,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9399999999998,72.996,7.445071062271063


A query to categorize each invoice as either an album purchase or not, and to calculate summary statistics - number of invoices, percentage of invoices. 

In [214]:
%%sql
with album_tracks as
(
    select
        t.track_id,
        a.album_id
    from track t
    inner join album a
    on t.album_id = a.album_id
),
album_track_count as
(
    select 
        album_id,
        COUNT(track_id) as number_of_tracks
    from album_tracks
    group by album_id
),
album_purchases as
(
    select
    *
    from album_track_count at
    inner join 
    (
        select 
            i.invoice_id,
            t.album_id,
            COUNT(t.track_id) as purchased
        from invoice_line i
        inner join album_tracks t 
        on t.track_id = i.track_id
        group by invoice_id,album_id
    ) i
    on at.album_id = i.album_id
),
purchase_flag as
(
    select
        invoice_id,
        SUM(
            case
                when number_of_tracks = purchased then 0
                else 1
            end
        ) as total
    from album_purchases
    group by invoice_id
)

select 
    buying_habbit,
    SUM(number) as number_of_invoices,
    CAST(SUM(number) as Float)/(select COUNT(DISTINCT(invoice_id)) from invoice) * 100 as percentage
from 
(
    select 
        case
            when total < 1 then "Buys_albums"
            else "Buys_individuals"
        end as buying_habbit,
        case 
            when total > 0 then 1
            else 0
        end as flag,
        COUNT(invoice_id) as number
    from purchase_flag
    group by total
)
group by buying_habbit

Done.


buying_habbit,number_of_invoices,percentage
Buys_albums,114,18.566775244299674
Buys_individuals,500,81.43322475570032


Based on the summary, we can conclude, people prefer buying individual songs rather than entire albums - 81% of invoices. Only 18% of the invoices are only albums.

A query to find the most popular artist among the music playlists

In [238]:
%%sql
with track_artist as
(
    select
    t.track_id,
    a.album_id,
    at.name as artist_name
    from track t
    inner join album a on a.album_id = t.album_id
    inner join artist at on a.artist_id = at.artist_id
)

select 
    artist_name,
    COUNT(DISTINCT(playlist_id)) as number_of_playlists
from playlist_track pt
natural join track_artist
group by artist_name
order by number_of_playlists desc;

Done.


artist_name,number_of_playlists
Eugene Ormandy,7
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Berliner Philharmoniker & Herbert Von Karajan,6
English Concert & Trevor Pinnock,6
The King's Singers,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5
Adrian Leaper & Doreen de Feis,5
Alberto Turco & Nova Schola Gregoriana,5
Antal Doráti & London Symphony Orchestra,5
Barry Wordsworth & BBC Concert Orchestra,5


A query to find the effect on popularity/sales by media-types protected and non-protected

In [257]:
%%sql
with media_groups as
(
    select 
        media_type_id,
        case
            when name like "protected%" then "protected"
            else "non-protected"
        end as type
    from media_type
),
track_media as
(
    select
        t.track_id,
        m.media_type_id,
        m.type
    from track t
    natural join media_groups m
),
invoice_tracks as
(
    select
        i.invoice_id,
        t.track_id,
        t.media_type_id,
        t.type
    from track_media t
    natural join invoice_line i
)

select
    type,
    COUNT(invoice_id) as number_of_purchases,
    CAST(COUNT(invoice_id) as Float)/(select COUNT(invoice_id) from invoice_line) * 100 as sales_percentage
from invoice_tracks
group by type

Done.


type,number_of_purchases,sales_percentage
non-protected,4315,90.70842968257305
protected,442,9.29157031742695
